In [2]:
from pathlib import Path
from datetime import datetime
import json
import pandas as pd
import re

In [7]:
def retrieve_game_duration(p2log):

    df = pd.DataFrame(columns=['session nb', 'game name', 'duration'])

    with open(p2log, 'r') as f:
        session_nb = 0
        while True:
            line = f.readline()
            
            # new session
            # 2023-11-13 17:20:41          New Session: 2023-11-13 17:20:41   
            regex = "^New Session: (?P<dt>\d\d\d\d-\d\d-\d\d\s\d\d:\d\d:\d\d)\s*$"
            matches = re.match(regex, line)
            if matches:
                session_nb += 1
                previous_game_name = None
                previous_game_start = None
                # print(session_nb, line)
            
            # get new game name and start time 
            # 2023-10-21 20:02:28   update info for gameidx: 20 and name Sound Effects
            # 2023-10-21 20:06:14   update info for gameidx: 6 and name Pillars
            regex = r"^(?P<dt>\d\d\d\d-\d\d-\d\d\s\d\d:\d\d:\d\d)\s*update\sinfo\sfor\sgameidx\:\s\d*\sand\sname\s(?P<game_name>[a-zA-Z\s]*)$"
            matches = re.match(regex, line, flags=re.MULTILINE)
            if matches:
                # print(line[:-1])
                new_game_name = matches.group('game_name').strip('\n')
                new_game_start = datetime.strptime(matches.group('dt'), '%Y-%m-%d %H:%M:%S')
                # print(new_game_start, new_game_name)
                
                # get finished game duration
                if previous_game_name is not None:
                    # print(previous_game_start, new_game_start)
                    duration = (new_game_start - previous_game_start).total_seconds()
                    # print(session_nb, previous_game_name, duration)
                    row = pd.DataFrame({'session nb': session_nb, 'game name': previous_game_name, 'duration': duration}, index=[0])
                    df = pd.concat([df, row], ignore_index=True) if len(df) > 0 else row
                    # display(df)
                previous_game_name = new_game_name
                previous_game_start = new_game_start
    
            if line == '':
                break
    return df
            

In [8]:
p2log = Path('../logs/_short_2023.log').resolve()
assert p2log.exists()
df_2023 = retrieve_game_duration(p2log)
df_2023.shape

(39, 3)

In [9]:
df_2023

,session nb,game name,duration
0,1,Sound Effects,226.0
1,1,Pillars,297.0
2,1,Sure Ding,320.0
3,1,Everybody Dies,1035.0
4,1,Stunt Doubles,592.0
5,1,Deaf Replay,470.0
6,1,Questions Only,390.0
7,2,Stunt Doubles,1.0
8,2,Forward Reverse,0.0
9,2,Chain Death,1.0


In [10]:
p2log = Path('../logs/_short.log').resolve()
assert p2log.exists()
df_2024 = retrieve_game_duration(p2log)
df_2024.shape

(15, 3)

In [11]:
all_games = pd.concat([df_2023, df_2024], ignore_index=True)
valid_games = all_games[all_games['duration'] > 30]
valid_games.shape

(30, 3)

In [12]:
valid_games.groupby('game name')[['duration']].mean()

,duration
game name,
A Date with Me,212.000000
Da Doo Ron Ron,438.000000
Deaf Replay,470.000000
Double Reverse Alphabet,395.000000
Emotion Roller Coaster,248.000000
Everybody Dies,763.000000
Everyone Dies,620.000000
Forward Reverse,345.500000
Genres,495.000000


In [13]:
valid_games.groupby('game name')[['duration']].mean().to_dict()['duration']

{'A Date with Me': 212.0,
 'Da Doo Ron Ron': 438.0,
 'Deaf Replay': 470.0,
 'Double Reverse Alphabet': 395.0,
 'Emotion Roller Coaster': 248.0,
 'Everybody Dies': 763.0,
 'Everyone Dies': 620.0,
 'Forward Reverse': 345.5,
 'Genres': 495.0,
 'Highlander': 830.0,
 'More or Less': 237.0,
 'No Laughing': 278.5,
 'One Three Five': 263.0,
 'Pillars': 317.6666666666667,
 'Puppets': 201.5,
 'Questions Only': 373.5,
 'Sound Effects': 226.0,
 'Stunt Doubles': 413.0,
 'Suitcase': 669.0,
 'Sure Ding': 502.5}

In [14]:
p2game_durations = p2log.parent / 'game_durations.json'

with open(p2game_durations, 'w') as f:
    json.dump(valid_games.groupby('game name')[['duration']].mean().to_dict()['duration'], f, indent=4)